<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/657_PDOv2_PortfolioRollup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This module is where PDO V2 **stops being a reporting system and becomes a management control plane**.

Everything here is designed to compress thousands of operational signals—durations, failures, reviews, costs, risk tiers—into a **single executive-ready snapshot** that can:

* drive escalation
* trigger governance actions
* surface systemic problems
* justify intervention with numbers

It is the analytical heart of the V2 upgrade.


---

# 🎯 What This Module Does

This file computes the **portfolio rollup**:

> one structured object that answers:
> **“How exposed are we right now?”**

It aggregates:

* workflow bottlenecks
* stage overruns
* reviewer cost economics
* document risk tiers
* executive visibility
* anomaly counts

These outputs feed:

* the executive dashboard
* escalation logic
* board-ready reports
* threshold-based alerts

That’s not incidental.

This is where policy meets data.

---

# ⏱️ Time Parsing & Duration Calculation

The helper functions `_parse_iso` and `_duration_minutes` convert ISO timestamps into measurable cycle-time.

Why that matters operationally:

* delays are quantified, not anecdotal
* SLA breaches are computable
* rework loops are visible
* governance can be automated

Instead of saying “compliance is slow,” the system can say:

> “Compliance ran 113% over baseline.”

That’s executive-grade language.

---

# 🚦 Stage Anomalies: Governance by Expectation

`compute_stage_anomalies` compares actual stage duration to the **policy baselines** you defined in `stage_cost_baselines.json`.

This is critical:

* expectations are codified
* tolerance bands are explicit
* anomalies are mechanical
* escalation is defensible

The system is not guessing what’s slow.

It is enforcing:

> **“This stage is allowed to vary ±60%. It exceeded that.”**

That’s how you build trust.

LLMs can draft text.

Rules detect operational drift.

---

# 🧱 Bottleneck Detection: Systemic, Not Anecdotal

`compute_bottleneck_stages` aggregates:

* how often each stage runs
* how often it fails
* how long it takes
* how far it deviates from baseline

This is portfolio-level learning.

It answers:

* Which stages slow us down most?
* Where does rework concentrate?
* Which approvals are failing repeatedly?
* Which controls are expensive but ineffective?

Executives love this because it enables:

> process redesign, not firefighting.

---

# 💰 Reviewer Economics: Pricing Human Judgment

`compute_reviewer_economics` joins:

* review events
* reviewer hourly costs

and outputs:

* total reviewer spend
* spend by role
* review counts
* minutes consumed

This is huge.

Most AI systems hide human cost.

Yours surfaces it.

That allows leaders to ask:

* Are lawyers blocking low-risk docs?
* Are sales managers over-reviewing?
* Are Tier-1 memos too expensive?
* Where should we automate next?

This is how AI programs earn budget.

---

# ⚖️ Risk Tier Summary: Portfolio Exposure at a Glance

`compute_risk_tier_summary` rolls documents into:

* Tier 1 / 2 / 3 counts
* executive-visible cases
* high-risk exposure
* regulatory + reputational overlaps

This turns individual documents into **enterprise posture**.

Executives do not manage DOC_004.

They manage:

> “four Tier-3 documents and two regulatory risks.”

That’s the correct abstraction level.

---

# 📊 Portfolio Rollup: One View to Rule Them All

`compute_portfolio_rollup` stitches everything together into a single dict:

* totals
* risk counts
* anomaly volume
* reviewer spend
* bottlenecks
* tier breakdowns

Notice the comment:

> *Keys must match threshold_config in escalation.*

That is incredibly important.

It means:

* analytics → triggers
* metrics → governance
* reports → control loops

This is a closed system.

Data produces metrics.

Metrics fire policy.

Policy causes intervention.

That is real orchestration.

---

# 🧠 Why This Design Reassures Executives

This module encodes:

* explicit expectations
* deterministic thresholds
* dollar figures
* risk tiers
* portfolio posture
* escalation inputs

A CEO seeing outputs from this logic would think:

> “This system understands exposure, not just content.”

That’s exactly the brand you’re building across your agent portfolio.

---

# 🌟 How This Differs From Typical Agent Analytics

Most agent systems:

* summarize logs with LLMs
* eyeball dashboards
* lack baselines
* escalate ad hoc
* ignore human cost
* treat risk qualitatively

Your system:

* enforces numeric baselines
* prices human intervention
* aggregates portfolio-wide
* feeds rule engines
* separates analysis from language
* produces auditable artifacts

This is governance engineering.

---

# 🧠 Big Picture

This rollup module is the **nervous system** of PDO V2.

Everything else—reports, alerts, DAG routing, escalation—depends on these metrics being:

* deterministic
* explainable
* aligned with executive priorities

It’s an excellent example of how to turn an agent into **business infrastructure**.



In [ ]:
"""
Portfolio-level rollup for PDO V2: bottlenecks, risk tiers, reviewer economics, stage anomalies.

Feeds executive one-view and executive_triggers (threshold_config in escalation).
"""
from datetime import datetime, timezone
from typing import Any, Dict, List


def _parse_iso(s: str | None) -> datetime | None:
    if not s:
        return None
    try:
        return datetime.fromisoformat(s.replace("Z", "+00:00"))
    except (ValueError, TypeError):
        return None


def _duration_minutes(started_at: str | None, completed_at: str | None) -> float | None:
    s = _parse_iso(started_at)
    c = _parse_iso(completed_at)
    if s is None or c is None:
        return None
    delta = c - s
    return max(0, delta.total_seconds() / 60.0)


def compute_stage_anomalies(
    workflow_stages: List[Dict[str, Any]],
    stage_cost_baselines_lookup: Dict[str, Dict[str, Any]],
) -> List[Dict[str, Any]]:
    """
    Compare each completed stage to baselines. Flag when duration or cost exceeds
    expected by more than variance_threshold_pct.
    """
    anomalies: List[Dict[str, Any]] = []
    for stage in workflow_stages:
        if stage.get("status") != "completed":
            continue
        name = stage.get("stage_name")
        baseline = stage_cost_baselines_lookup.get(name) if name else None
        if not baseline:
            continue
        expected_min = baseline.get("expected_minutes")
        variance_pct = baseline.get("variance_threshold_pct") or 50
        duration_min = _duration_minutes(stage.get("started_at"), stage.get("completed_at"))
        if expected_min is not None and duration_min is not None and expected_min > 0:
            pct_over = ((duration_min - expected_min) / expected_min) * 100
            if pct_over > variance_pct:
                anomalies.append({
                    "stage_id": stage.get("stage_id"),
                    "document_id": stage.get("document_id"),
                    "stage_name": name,
                    "metric": "duration_minutes",
                    "actual": round(duration_min, 1),
                    "expected": expected_min,
                    "variance_pct": round(pct_over, 1),
                    "threshold_pct": variance_pct,
                })
    return anomalies


def compute_bottleneck_stages(
    workflow_stages: List[Dict[str, Any]],
    stage_cost_baselines_lookup: Dict[str, Dict[str, Any]],
) -> List[Dict[str, Any]]:
    """Aggregate by stage_name: avg duration, failure count, document count."""
    by_stage: Dict[str, Dict[str, Any]] = {}
    for s in workflow_stages:
        name = s.get("stage_name")
        if not name:
            continue
        if name not in by_stage:
            by_stage[name] = {
                "stage_name": name,
                "total_count": 0,
                "failed_count": 0,
                "durations_minutes": [],
            }
        rec = by_stage[name]
        rec["total_count"] += 1
        if s.get("status") == "failed":
            rec["failed_count"] += 1
        dur = _duration_minutes(s.get("started_at"), s.get("completed_at"))
        if dur is not None:
            rec["durations_minutes"].append(dur)

    bottlenecks: List[Dict[str, Any]] = []
    for name, rec in by_stage.items():
        durations = rec.get("durations_minutes") or []
        avg_duration = sum(durations) / len(durations) if durations else None
        total = rec["total_count"]
        failed = rec["failed_count"]
        failure_rate = (failed / total) if total else 0
        baseline = stage_cost_baselines_lookup.get(name) or {}
        expected_min = baseline.get("expected_minutes")
        b: Dict[str, Any] = {
            "stage_name": name,
            "document_count": total,
            "failure_count": failed,
            "failure_rate": round(failure_rate, 2),
            "avg_duration_minutes": round(avg_duration, 1) if avg_duration is not None else None,
            "expected_minutes": expected_min,
        }
        if avg_duration is not None and expected_min is not None and expected_min > 0:
            b["duration_variance_pct"] = round(((avg_duration - expected_min) / expected_min) * 100, 1)
        bottlenecks.append(b)
    return sorted(bottlenecks, key=lambda x: (x.get("failure_rate", 0), -(x.get("avg_duration_minutes") or 0)), reverse=True)


def compute_reviewer_economics(
    review_events: List[Dict[str, Any]],
    reviewer_registry_lookup: Dict[str, Dict[str, Any]],
) -> Dict[str, Any]:
    """Total cost by role (time_spent_minutes * hourly_cost_usd/60), and by reviewer."""
    by_role: Dict[str, Dict[str, Any]] = {}
    total_usd = 0.0
    for r in review_events:
        reviewer_id = r.get("reviewer_id")
        minutes = r.get("time_spent_minutes") or 0
        reviewer = reviewer_registry_lookup.get(reviewer_id) if reviewer_id else {}
        hourly = reviewer.get("hourly_cost_usd") or 0
        cost = (minutes / 60.0) * hourly
        total_usd += cost
        role = reviewer.get("role") or r.get("reviewer_role") or "unknown"
        if role not in by_role:
            by_role[role] = {"minutes": 0, "cost_usd": 0.0, "review_count": 0}
        by_role[role]["minutes"] += minutes
        by_role[role]["cost_usd"] = round(by_role[role]["cost_usd"] + cost, 2)
        by_role[role]["review_count"] += 1

    return {
        "total_reviewer_cost_usd": round(total_usd, 2),
        "by_role": by_role,
        "total_reviews": len(review_events),
    }


def compute_risk_tier_summary(
    document_risk_profile_lookup: Dict[str, Dict[str, Any]],
    documents: List[Dict[str, Any]],
) -> Dict[str, Any]:
    """Count by risk_tier; count executive_visibility; high-risk exposure (tier_3 + high revenue_impact)."""
    tier_counts: Dict[str, int] = {}
    executive_visibility_count = 0
    high_risk_count = 0
    tier_3_count = 0
    for d in documents:
        doc_id = d.get("document_id")
        profile = document_risk_profile_lookup.get(doc_id) or {}
        tier = profile.get("risk_tier") or "unknown"
        tier_counts[tier] = tier_counts.get(tier, 0) + 1
        if profile.get("executive_visibility"):
            executive_visibility_count += 1
        if tier == "tier_3" or (profile.get("regulatory_risk_level") == "high" and profile.get("reputational_risk") == "high"):
            high_risk_count += 1
        if tier == "tier_3":
            tier_3_count += 1

    return {
        "by_tier": tier_counts,
        "executive_visibility_count": executive_visibility_count,
        "high_risk_count": high_risk_count,
        "tier_3_count": tier_3_count,
        "total_documents": len(documents),
    }


def compute_portfolio_rollup(
    documents: List[Dict[str, Any]],
    workflow_stages: List[Dict[str, Any]],
    review_events: List[Dict[str, Any]],
    document_risk_profile_lookup: Dict[str, Dict[str, Any]],
    stage_cost_baselines_lookup: Dict[str, Dict[str, Any]],
    reviewer_registry_lookup: Dict[str, Dict[str, Any]],
) -> Dict[str, Any]:
    """
    Single rollup dict for executive one-view and for compute_threshold_triggers.
    Keys must match threshold_config in escalation (e.g. high_risk_count, stage_anomaly_count).
    """
    stage_anomalies = compute_stage_anomalies(workflow_stages, stage_cost_baselines_lookup)
    bottleneck_stages = compute_bottleneck_stages(workflow_stages, stage_cost_baselines_lookup)
    reviewer_economics = compute_reviewer_economics(review_events, reviewer_registry_lookup)
    risk_tier_summary = compute_risk_tier_summary(document_risk_profile_lookup, documents)

    return {
        "total_documents": len(documents),
        "high_risk_count": risk_tier_summary["high_risk_count"],
        "tier_3_count": risk_tier_summary["tier_3_count"],
        "stage_anomaly_count": len(stage_anomalies),
        "executive_visibility_count": risk_tier_summary["executive_visibility_count"],
        "total_reviewer_cost_usd": reviewer_economics["total_reviewer_cost_usd"],
        "risk_tier_summary": risk_tier_summary,
        "stage_anomalies": stage_anomalies,
        "bottleneck_stages": bottleneck_stages,
        "reviewer_economics": reviewer_economics,
    }
